In [1]:
import pandas as pd
import time
from collections import defaultdict

In [2]:
%%time
train = pd.read_csv('./train_preprocess1.csv',
                   dtype={
                       'C14_fix':str,
                       'C17_fix':str,
                       'C19_fix':str,
                       'C20_fix':str
                   })
train.shape

CPU times: user 2min 18s, sys: 48.1 s, total: 3min 6s
Wall time: 3min 19s


(40428967, 24)

In [3]:
train.dtypes

id                   float64
click                  int64
C1                     int64
banner_pos             int64
site_id_fix           object
site_domain_fix       object
site_category_fix     object
app_id_fix            object
app_domain_fix        object
app_category_fix      object
device_id_fix         object
device_ip_fix         object
device_model_fix      object
device_type            int64
device_conn_type       int64
C14_fix               object
C15                    int64
C16                    int64
C17_fix               object
C18                    int64
C19_fix               object
C20_fix               object
C21                    int64
hour_fix               int64
dtype: object

In [4]:
f_cols = train.columns.tolist()
f_cols.remove('id')
f_cols.remove('click')

In [7]:
%%time
train[f_cols].nunique()

id                   40428967
click                       2
C1                          7
banner_pos                  7
site_id_fix              3478
site_domain_fix          4566
site_category_fix          22
app_id_fix               4905
app_domain_fix            296
app_category_fix           28
device_id_fix          655199
device_ip_fix         2268793
device_model_fix         6529
device_type                 5
device_conn_type            4
C14_fix                  2415
C15                         8
C16                         9
C17_fix                   426
C18                         4
C19_fix                    68
C20_fix                   167
C21                        60
hour_fix                   24
dtype: int64

## 點擊率計算

In [5]:
def category_ctr(col):
    all_cnt = train[col].value_counts().to_dict()
    cli_cnt = train[col][train['click']==1].value_counts().to_dict()

    for cate in all_cnt.keys():
        if cate != 'DNP':
            cate_ctr['column'].append(col)
            cate_ctr['category'].append(cate)
            cate_ctr['impressions'].append(all_cnt[cate])
            cate_ctr['clicks'].append(cli_cnt[cate])
            cate_ctr['ctr'].append(round(cli_cnt[cate]/all_cnt[cate], 4))
    print(f'[ {col} ] CTR calculate complete')

In [6]:
%%time
cate_ctr = defaultdict(list)

for col in f_cols:
    category_ctr(col)

[ C1 ] CTR calculate complete
[ banner_pos ] CTR calculate complete
[ site_id_fix ] CTR calculate complete
[ site_domain_fix ] CTR calculate complete
[ site_category_fix ] CTR calculate complete
[ app_id_fix ] CTR calculate complete
[ app_domain_fix ] CTR calculate complete
[ app_category_fix ] CTR calculate complete
[ device_id_fix ] CTR calculate complete
[ device_ip_fix ] CTR calculate complete
[ device_model_fix ] CTR calculate complete
[ device_type ] CTR calculate complete
[ device_conn_type ] CTR calculate complete
[ C14_fix ] CTR calculate complete
[ C15 ] CTR calculate complete
[ C16 ] CTR calculate complete
[ C17_fix ] CTR calculate complete
[ C18 ] CTR calculate complete
[ C19_fix ] CTR calculate complete
[ C20_fix ] CTR calculate complete
[ C21 ] CTR calculate complete
[ hour_fix ] CTR calculate complete
CPU times: user 1min 48s, sys: 8.5 s, total: 1min 57s
Wall time: 1min 57s


In [7]:
df_cate_ctr = pd.DataFrame(cate_ctr)
df_cate_ctr.shape

(2947007, 5)

In [8]:
df_cate_ctr.head()

,column,category,impressions,clicks,ctr
0,C1,1005,37140632,6289056,0.1693
1,C1,1002,2220812,467995,0.2107
2,C1,1010,903457,86023,0.0952
3,C1,1012,113512,19580,0.1725
4,C1,1007,35304,1392,0.0394


In [9]:
df_cate_ctr.describe().round(2)

,impressions,clicks,ctr
count,2947007.00,2947007.00,2947007.00
mean,296.37,51.25,0.63
std,68255.41,11827.33,0.35
min,1.00,1.00,0.00
25%,1.00,1.00,0.33
50%,2.00,1.00,0.50
75%,6.00,2.00,1.00
max,38136554.00,6318889.00,1.00


In [10]:
df_cate_ctr[df_cate_ctr['column']=='C1'][['impressions', 'ctr']].mean().round(2)

impressions    5775566.71
ctr                  0.12
dtype: float64

In [11]:
df_cate_ctr[df_cate_ctr['column']=='device_id_fix'][['impressions', 'ctr']].mean().round(2)

impressions    55.03
ctr             0.74
dtype: float64

In [12]:
df_cate_ctr[df_cate_ctr['column']=='device_ip_fix'][['impressions', 'ctr']].mean().round(2)

impressions    12.78
ctr             0.60
dtype: float64

In [14]:
df_cate_ctr[df_cate_ctr['impressions'] >= 296][['impressions', 'ctr']].describe().round(2)

,impressions,ctr
count,20760.00,20760.00
mean,41033.36,0.16
std,812222.35,0.10
min,296.00,0.00
25%,408.00,0.08
50%,605.00,0.17
75%,1805.25,0.22
max,38136554.00,0.96


In [18]:
df_cate_ctr[(df_cate_ctr['impressions'] >= 41033) & (df_cate_ctr['ctr'] >= 0.16)]['category'].nunique()

439

In [19]:
cate_del = df_cate_ctr[(df_cate_ctr['impressions'] < 41033) | (df_cate_ctr['ctr'] < 0.16)]
cate_del.shape

(2946557, 5)

In [20]:
cate_del.head(10)

,column,category,impressions,clicks,ctr
2,C1,1010,903457,86023,0.0952
4,C1,1007,35304,1392,0.0394
5,C1,1001,9463,316,0.0334
6,C1,1008,5787,704,0.1217
10,banner_pos,2,13001,1550,0.1192
11,banner_pos,4,7704,1428,0.1854
12,banner_pos,5,5778,702,0.1215
13,banner_pos,3,2035,372,0.1828
14,site_id_fix,85f751fd,14596137,1734407,0.1188
19,site_id_fix,5b4d2eda,771360,100287,0.1300


In [21]:
cate_del_map = defaultdict(list)

for col in cate_del['column'].unique():
    cate_del_map[col] = cate_del[cate_del['column'] == col]['category'].unique().tolist()

In [22]:
cate_del_map

defaultdict(list,
            {'C1': [1010, 1007, 1001, 1008],
             'banner_pos': [2, 4, 5, 3],
             'site_id_fix': ['85f751fd',
              '5b4d2eda',
              '856e6d3f',
              'a7853007',
              'b7e9786d',
              '6399eda6',
              '5bcf81a2',
              '57ef2c87',
              '83a0ad1a',
              'e4d8dd7b',
              '12fb4121',
              'e5c60a05',
              '9a977531',
              'bb4524e7',
              '5114c672',
              'e3c09f3a',
              'ce3307ec',
              '6ec06dbd',
              '0eb72673',
              '92c7cbe7',
              '16c73019',
              '89a490f5',
              '9e8cf15d',
              'ba9aee1b',
              '4e7614cf',
              '7dd19f44',
              '26fa1946',
              '43d6df75',
              'b99a2c43',
              'c1fcc9a1',
              'fec6e632',
              '7294ea0f',
              '517b8671',
              '222acd54

In [23]:
# DNP mapping
def dnp_mapping(col):
    # col = 'site_id'
    dnp_map = {cate:'DNP' for cate in cate_del_map[col]}
    train[f'{col}_fix'] = train[col].map(dnp_map).fillna(train[col].astype(str))

    print(f'[ {col} ] mapping complete')

In [24]:
%%time
for col in cate_del['column'].unique():
    dnp_mapping(col)

[ C1 ] mapping complete
[ banner_pos ] mapping complete
[ site_id_fix ] mapping complete
[ site_domain_fix ] mapping complete
[ site_category_fix ] mapping complete
[ app_id_fix ] mapping complete
[ app_domain_fix ] mapping complete
[ app_category_fix ] mapping complete
[ device_id_fix ] mapping complete
[ device_ip_fix ] mapping complete
[ device_model_fix ] mapping complete
[ device_type ] mapping complete
[ device_conn_type ] mapping complete
[ C14_fix ] mapping complete
[ C15 ] mapping complete
[ C16 ] mapping complete
[ C17_fix ] mapping complete
[ C18 ] mapping complete
[ C19_fix ] mapping complete
[ C20_fix ] mapping complete
[ C21 ] mapping complete
[ hour_fix ] mapping complete
CPU times: user 6min 58s, sys: 41.5 s, total: 7min 39s
Wall time: 7min 49s


In [25]:
col_sel = ['id', 'click']
for col in cate_del['column'].unique():
    col_sel.append(f'{col}_fix')
col_sel

['id',
 'click',
 'C1_fix',
 'banner_pos_fix',
 'site_id_fix_fix',
 'site_domain_fix_fix',
 'site_category_fix_fix',
 'app_id_fix_fix',
 'app_domain_fix_fix',
 'app_category_fix_fix',
 'device_id_fix_fix',
 'device_ip_fix_fix',
 'device_model_fix_fix',
 'device_type_fix',
 'device_conn_type_fix',
 'C14_fix_fix',
 'C15_fix',
 'C16_fix',
 'C17_fix_fix',
 'C18_fix',
 'C19_fix_fix',
 'C20_fix_fix',
 'C21_fix',
 'hour_fix_fix']

In [26]:
%%time
train[col_sel].to_csv('./train_preprocess2.csv', index=False)

CPU times: user 6min 46s, sys: 5min 39s, total: 12min 26s
Wall time: 15min 20s
